# 📊 Dashboard Interativo - Big Data Finance

Este notebook demonstra como criar dashboards interativos para análise financeira usando Plotly, Streamlit e outras ferramentas de visualização.

## Objetivos
- Criar visualizações interativas com Plotly
- Implementar dashboard web com Streamlit
- Demonstrar gráficos financeiros especializados
- Integrar análises de ML e sentimentos
- Criar relatórios executivos
- Monitoramento em tempo real

**Autor:** Ricardo Areas (Visualização) & Equipe Big Data Finance  
**Gestão:** Fabio  
**Infraestrutura:** Ana Luiza Pazze  
**Data:** 2024

In [ ]:
# Imports necessários
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Adicionar src ao path
sys.path.append('../src')

# Imports para visualização
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

# Imports dos módulos do projeto
from visualization.dashboard import FinancialDashboard
from visualization.charts import FinancialCharts
from data_collection.yahoo_finance_collector import YahooFinanceCollector

print("✅ Imports realizados com sucesso!")
print("📊 Plotly configurado para notebook")

## 1. 📊 Carregamento dos Dados

Vamos carregar os dados processados e resultados dos modelos.

In [ ]:
# Carregar dados processados
print("📊 Carregando dados para visualização...")

# Verificar dados disponíveis
raw_dir = '../data/raw'
results_dir = '../data/results'

# Carregar dados de ações
if os.path.exists(raw_dir):
    stock_files = [f for f in os.listdir(raw_dir) if f.startswith('stock_data_') and f.endswith('.csv')]
    if stock_files:
        latest_file = sorted(stock_files)[-1]
        df_stocks = pd.read_csv(f"{raw_dir}/{latest_file}")
        print(f"📈 Dados de ações carregados: {latest_file}")
    else:
        print("⚠️ Dados de ações não encontrados. Criando dados sintéticos...")
        # Criar dados sintéticos
        dates = pd.date_range(start='2023-01-01', end='2024-01-01', freq='D')
        symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN']
        
        data = []
        for symbol in symbols:
            base_price = np.random.uniform(100, 300)
            for i, date in enumerate(dates):
                price_change = np.random.normal(0, 0.02)
                base_price *= (1 + price_change)
                
                data.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'symbol': symbol,
                    'open': base_price * np.random.uniform(0.98, 1.02),
                    'high': base_price * np.random.uniform(1.00, 1.05),
                    'low': base_price * np.random.uniform(0.95, 1.00),
                    'close': base_price,
                    'volume': np.random.randint(1000000, 10000000)
                })
        
        df_stocks = pd.DataFrame(data)
        print("✅ Dados sintéticos criados")
else:
    print("❌ Diretório de dados não encontrado")
    df_stocks = pd.DataFrame()

# Converter data
if not df_stocks.empty:
    df_stocks['date'] = pd.to_datetime(df_stocks['date'])
    print(f"📊 Total de registros: {len(df_stocks):,}")
    print(f"📅 Período: {df_stocks['date'].min()} até {df_stocks['date'].max()}")
    print(f"📈 Símbolos: {list(df_stocks['symbol'].unique())}")

# Carregar resultados de ML se disponíveis
ml_results = {}
if os.path.exists(results_dir):
    result_files = {
        'regression': 'regression_results.csv',
        'classification': 'classification_results.csv',
        'feature_importance': 'feature_importance.csv',
        'sentiment': 'sentiment_analysis.csv'
    }
    
    for key, filename in result_files.items():
        filepath = f"{results_dir}/{filename}"
        if os.path.exists(filepath):
            ml_results[key] = pd.read_csv(filepath)
            print(f"🤖 {key.title()} results carregados")

print(f"\n✅ Carregamento concluído!")
display(df_stocks.head() if not df_stocks.empty else "Nenhum dado disponível")

## 2. 📈 Gráficos de Preços Interativos

Vamos criar gráficos interativos de preços usando Plotly.

In [ ]:
# Inicializar dashboard
dashboard = FinancialDashboard()

if not df_stocks.empty:
    print("📈 Criando gráficos de preços interativos...")
    
    # Selecionar um símbolo para demonstração
    symbol = df_stocks['symbol'].iloc[0]
    symbol_data = df_stocks[df_stocks['symbol'] == symbol].copy().sort_values('date')
    
    print(f"📊 Analisando {symbol}...")
    
    # Gráfico de linha simples
    fig_line = dashboard.create_price_chart(symbol_data, chart_type='line')
    fig_line.update_layout(
        title=f"📈 Evolução do Preço - {symbol}",
        height=500
    )
    fig_line.show()
    
    # Gráfico candlestick
    print(f"\n🕯️ Criando gráfico candlestick para {symbol}...")
    fig_candle = dashboard.create_candlestick_chart(symbol_data)
    fig_candle.update_layout(
        title=f"🕯️ Gráfico Candlestick - {symbol}",
        height=600
    )
    fig_candle.show()
    
else:
    print("❌ Não há dados disponíveis para criar gráficos")

## 3. 📊 Comparação Multi-Ativos

Vamos comparar múltiplos ativos em um único gráfico.

In [ ]:
if not df_stocks.empty:
    print("📊 Criando comparação multi-ativos...")
    
    # Normalizar preços para comparação
    normalized_data = []
    
    for symbol in df_stocks['symbol'].unique():
        symbol_data = df_stocks[df_stocks['symbol'] == symbol].copy().sort_values('date')
        
        # Normalizar preços (base 100)
        first_price = symbol_data['close'].iloc[0]
        symbol_data['normalized_price'] = (symbol_data['close'] / first_price) * 100
        symbol_data['returns'] = symbol_data['close'].pct_change() * 100
        
        normalized_data.append(symbol_data)
    
    df_normalized = pd.concat(normalized_data, ignore_index=True)
    
    # Gráfico de performance normalizada
    fig_multi = px.line(
        df_normalized, 
        x='date', 
        y='normalized_price', 
        color='symbol',
        title='📊 Performance Comparativa (Base 100)',
        labels={
            'normalized_price': 'Preço Normalizado',
            'date': 'Data',
            'symbol': 'Símbolo'
        }
    )
    
    fig_multi.update_layout(
        height=500,
        hovermode='x unified',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    fig_multi.show()
    
    # Matriz de correlação interativa
    print("\n🔗 Criando matriz de correlação...")
    
    # Pivot para matriz de correlação
    pivot_data = df_stocks.pivot(index='date', columns='symbol', values='close')
    correlation_matrix = pivot_data.corr()
    
    fig_corr = dashboard.create_correlation_heatmap(correlation_matrix)
    fig_corr.update_layout(
        title="🔗 Matriz de Correlação dos Ativos",
        height=500
    )
    fig_corr.show()
    
    print("\n📊 Estatísticas de Correlação:")
    print(correlation_matrix.round(3))

else:
    print("❌ Não há dados disponíveis para comparação")

## 4. 📈 Análise de Volatilidade

Vamos criar visualizações para análise de volatilidade.

In [ ]:
if not df_stocks.empty:
    print("📈 Criando análise de volatilidade...")
    
    # Calcular volatilidade para cada ativo
    volatility_data = []
    
    for symbol in df_stocks['symbol'].unique():
        symbol_data = df_stocks[df_stocks['symbol'] == symbol].copy().sort_values('date')
        
        # Calcular retornos e volatilidade
        symbol_data['returns'] = symbol_data['close'].pct_change()
        symbol_data['volatility_30d'] = symbol_data['returns'].rolling(window=30).std() * np.sqrt(252) * 100
        symbol_data['volatility_7d'] = symbol_data['returns'].rolling(window=7).std() * np.sqrt(252) * 100
        
        volatility_data.append(symbol_data)
    
    df_volatility = pd.concat(volatility_data, ignore_index=True)
    
    # Gráfico de volatilidade temporal
    fig_vol = make_subplots(
        rows=2, cols=1,
        subplot_titles=('📊 Preços', '📈 Volatilidade (30 dias)'),
        vertical_spacing=0.1
    )
    
    # Adicionar preços
    for symbol in df_volatility['symbol'].unique():
        symbol_data = df_volatility[df_volatility['symbol'] == symbol]
        
        fig_vol.add_trace(
            go.Scatter(
                x=symbol_data['date'],
                y=symbol_data['close'],
                name=f'{symbol} - Preço',
                line=dict(width=2)
            ),
            row=1, col=1
        )
        
        fig_vol.add_trace(
            go.Scatter(
                x=symbol_data['date'],
                y=symbol_data['volatility_30d'],
                name=f'{symbol} - Vol 30d',
                line=dict(width=2, dash='dot')
            ),
            row=2, col=1
        )
    
    fig_vol.update_layout(
        title="📈 Análise de Preços e Volatilidade",
        height=700,
        hovermode='x unified'
    )
    
    fig_vol.update_yaxes(title_text="Preço ($)", row=1, col=1)
    fig_vol.update_yaxes(title_text="Volatilidade (%)", row=2, col=1)
    fig_vol.update_xaxes(title_text="Data", row=2, col=1)
    
    fig_vol.show()
    
    # Box plot de volatilidade por ativo
    print("\n📊 Comparação de volatilidade por ativo...")
    
    fig_box = px.box(
        df_volatility.dropna(),
        x='symbol',
        y='volatility_30d',
        title='📊 Distribuição de Volatilidade por Ativo (30 dias)',
        labels={
            'volatility_30d': 'Volatilidade (%)',
            'symbol': 'Símbolo'
        }
    )
    
    fig_box.update_layout(height=400)
    fig_box.show()
    
    # Estatísticas de volatilidade
    vol_stats = df_volatility.groupby('symbol')['volatility_30d'].agg([
        'mean', 'std', 'min', 'max'
    ]).round(2)
    
    print("\n📊 Estatísticas de Volatilidade (30 dias):")
    display(vol_stats)

else:
    print("❌ Não há dados disponíveis para análise de volatilidade")

## 5. 🤖 Visualização dos Resultados de ML

Vamos visualizar os resultados dos modelos de Machine Learning.

In [ ]:
if ml_results:
    print("🤖 Visualizando resultados de Machine Learning...")
    
    # Performance dos modelos de regressão
    if 'regression' in ml_results:
        reg_results = ml_results['regression']
        
        fig_reg = make_subplots(
            rows=1, cols=2,
            subplot_titles=('📊 R² Score', '📊 MSE'),
            horizontal_spacing=0.1
        )
        
        # R² Score
        fig_reg.add_trace(
            go.Bar(
                x=reg_results['Modelo'],
                y=reg_results['Test R²'],
                name='Test R²',
                marker_color='lightblue'
            ),
            row=1, col=1
        )
        
        fig_reg.add_trace(
            go.Bar(
                x=reg_results['Modelo'],
                y=reg_results['Train R²'],
                name='Train R²',
                marker_color='darkblue',
                opacity=0.7
            ),
            row=1, col=1
        )
        
        # MSE
        fig_reg.add_trace(
            go.Bar(
                x=reg_results['Modelo'],
                y=reg_results['Test MSE'],
                name='Test MSE',
                marker_color='lightcoral',
                showlegend=False
            ),
            row=1, col=2
        )
        
        fig_reg.add_trace(
            go.Bar(
                x=reg_results['Modelo'],
                y=reg_results['Train MSE'],
                name='Train MSE',
                marker_color='darkred',
                opacity=0.7,
                showlegend=False
            ),
            row=1, col=2
        )
        
        fig_reg.update_layout(
            title="🤖 Performance dos Modelos de Regressão",
            height=500,
            barmode='group'
        )
        
        fig_reg.update_xaxes(tickangle=45)
        fig_reg.show()
    
    # Performance dos modelos de classificação
    if 'classification' in ml_results:
        class_results = ml_results['classification']
        
        fig_class = go.Figure()
        
        fig_class.add_trace(
            go.Bar(
                x=class_results['Modelo'],
                y=class_results['Test Accuracy'],
                name='Test Accuracy',
                marker_color='lightgreen'
            )
        )
        
        fig_class.add_trace(
            go.Bar(
                x=class_results['Modelo'],
                y=class_results['Train Accuracy'],
                name='Train Accuracy',
                marker_color='darkgreen',
                opacity=0.7
            )
        )
        
        fig_class.update_layout(
            title="🎯 Performance dos Modelos de Classificação",
            height=400,
            barmode='group',
            yaxis_title='Accuracy'
        )
        
        fig_class.update_xaxes(tickangle=45)
        fig_class.show()
    
    # Importância das features
    if 'feature_importance' in ml_results:
        importance_df = ml_results['feature_importance'].head(15)
        
        fig_importance = make_subplots(
            rows=1, cols=2,
            subplot_titles=('📊 Regressão', '🎯 Classificação'),
            horizontal_spacing=0.1
        )
        
        fig_importance.add_trace(
            go.Bar(
                y=importance_df['Feature'],
                x=importance_df['Importance_Regression'],
                orientation='h',
                name='Regressão',
                marker_color='skyblue'
            ),
            row=1, col=1
        )
        
        fig_importance.add_trace(
            go.Bar(
                y=importance_df['Feature'],
                x=importance_df['Importance_Classification'],
                orientation='h',
                name='Classificação',
                marker_color='coral'
            ),
            row=1, col=2
        )
        
        fig_importance.update_layout(
            title="🔍 Importância das Features (Top 15)",
            height=600,
            showlegend=False
        )
        
        fig_importance.show()

else:
    print("⚠️ Resultados de ML não disponíveis")
    print("💡 Execute primeiro o notebook 03_machine_learning_example.ipynb")

## 6. 💭 Visualização de Sentimentos

Vamos criar visualizações para análise de sentimentos.

In [ ]:
if 'sentiment' in ml_results:
    print("💭 Visualizando análise de sentimentos...")
    
    sentiment_df = ml_results['sentiment']
    
    # Distribuição de sentimentos
    fig_sent_dist = make_subplots(
        rows=1, cols=2,
        subplot_titles=('📊 Distribuição de Scores', '📊 Categorias'),
        specs=[[{"secondary_y": False}, {"type": "pie"}]]
    )
    
    # Histograma de scores
    fig_sent_dist.add_trace(
        go.Histogram(
            x=sentiment_df['score'],
            nbinsx=20,
            name='Scores',
            marker_color='lightblue',
            opacity=0.7
        ),
        row=1, col=1
    )
    
    # Linha vertical no zero
    fig_sent_dist.add_vline(
        x=0, line_dash="dash", line_color="red",
        annotation_text="Neutro", row=1, col=1
    )
    
    # Gráfico de pizza das categorias
    sentiment_counts = sentiment_df['label'].value_counts()
    
    fig_sent_dist.add_trace(
        go.Pie(
            labels=sentiment_counts.index,
            values=sentiment_counts.values,
            name="Categorias",
            marker_colors=['red', 'gray', 'green']
        ),
        row=1, col=2
    )
    
    fig_sent_dist.update_layout(
        title="💭 Análise de Sentimentos - Distribuição",
        height=400
    )
    
    fig_sent_dist.show()
    
    # Gráfico de barras dos textos com sentimentos
    print("\n📰 Sentimentos por texto...")
    
    # Ordenar por score
    sentiment_sorted = sentiment_df.sort_values('score')
    
    # Criar cores baseadas no sentimento
    colors = ['red' if score < -0.1 else 'green' if score > 0.1 else 'gray' 
              for score in sentiment_sorted['score']]
    
    fig_sent_texts = go.Figure()
    
    fig_sent_texts.add_trace(
        go.Bar(
            x=list(range(len(sentiment_sorted))),
            y=sentiment_sorted['score'],
            marker_color=colors,
            text=[text[:50] + '...' if len(text) > 50 else text 
                  for text in sentiment_sorted['text']],
            textposition='outside',
            hovertemplate='<b>Score:</b> %{y:.3f}<br>' +
                         '<b>Texto:</b> %{text}<br>' +
                         '<extra></extra>'
        )
    )
    
    fig_sent_texts.update_layout(
        title="📰 Sentimentos por Texto (Ordenado por Score)",
        height=500,
        xaxis_title="Índice do Texto",
        yaxis_title="Score de Sentimento",
        showlegend=False
    )
    
    fig_sent_texts.add_hline(
        y=0, line_dash="dash", line_color="black",
        annotation_text="Neutro"
    )
    
    fig_sent_texts.show()
    
    # Estatísticas de sentimento
    print("\n📊 Estatísticas de Sentimento:")
    stats = {
        'Média': sentiment_df['score'].mean(),
        'Mediana': sentiment_df['score'].median(),
        'Desvio Padrão': sentiment_df['score'].std(),
        'Mínimo': sentiment_df['score'].min(),
        'Máximo': sentiment_df['score'].max(),
        'Positivos': len(sentiment_df[sentiment_df['score'] > 0.1]),
        'Negativos': len(sentiment_df[sentiment_df['score'] < -0.1]),
        'Neutros': len(sentiment_df[abs(sentiment_df['score']) <= 0.1])
    }
    
    for key, value in stats.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.3f}")
        else:
            print(f"  {key}: {value}")

else:
    print("⚠️ Dados de sentimento não disponíveis")
    print("💡 Execute primeiro o notebook 03_machine_learning_example.ipynb")

## 7. 📊 Dashboard Executivo

Vamos criar um dashboard executivo com métricas principais.

In [ ]:
if not df_stocks.empty:
    print("📊 Criando Dashboard Executivo...")
    
    # Calcular métricas principais
    metrics = {}
    
    for symbol in df_stocks['symbol'].unique():
        symbol_data = df_stocks[df_stocks['symbol'] == symbol].copy().sort_values('date')
        
        # Calcular métricas
        current_price = symbol_data['close'].iloc[-1]
        first_price = symbol_data['close'].iloc[0]
        total_return = ((current_price - first_price) / first_price) * 100
        
        returns = symbol_data['close'].pct_change().dropna()
        volatility = returns.std() * np.sqrt(252) * 100
        
        max_price = symbol_data['close'].max()
        min_price = symbol_data['close'].min()
        
        avg_volume = symbol_data['volume'].mean()
        
        metrics[symbol] = {
            'current_price': current_price,
            'total_return': total_return,
            'volatility': volatility,
            'max_price': max_price,
            'min_price': min_price,
            'avg_volume': avg_volume
        }
    
    # Criar dashboard com subplots
    fig_dashboard = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            '💰 Preço Atual', '📈 Retorno Total (%)',
            '📊 Volatilidade (%)', '📊 Volume Médio',
            '🔝 Preço Máximo', '🔻 Preço Mínimo'
        ),
        vertical_spacing=0.08,
        horizontal_spacing=0.1
    )
    
    symbols = list(metrics.keys())
    
    # Preço atual
    fig_dashboard.add_trace(
        go.Bar(
            x=symbols,
            y=[metrics[s]['current_price'] for s in symbols],
            name='Preço Atual',
            marker_color='lightblue',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Retorno total
    returns_values = [metrics[s]['total_return'] for s in symbols]
    colors = ['green' if r > 0 else 'red' for r in returns_values]
    
    fig_dashboard.add_trace(
        go.Bar(
            x=symbols,
            y=returns_values,
            name='Retorno Total',
            marker_color=colors,
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Volatilidade
    fig_dashboard.add_trace(
        go.Bar(
            x=symbols,
            y=[metrics[s]['volatility'] for s in symbols],
            name='Volatilidade',
            marker_color='orange',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # Volume médio
    fig_dashboard.add_trace(
        go.Bar(
            x=symbols,
            y=[metrics[s]['avg_volume'] for s in symbols],
            name='Volume Médio',
            marker_color='purple',
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Preço máximo
    fig_dashboard.add_trace(
        go.Bar(
            x=symbols,
            y=[metrics[s]['max_price'] for s in symbols],
            name='Preço Máximo',
            marker_color='darkgreen',
            showlegend=False
        ),
        row=3, col=1
    )
    
    # Preço mínimo
    fig_dashboard.add_trace(
        go.Bar(
            x=symbols,
            y=[metrics[s]['min_price'] for s in symbols],
            name='Preço Mínimo',
            marker_color='darkred',
            showlegend=False
        ),
        row=3, col=2
    )
    
    fig_dashboard.update_layout(
        title="📊 Dashboard Executivo - Métricas Principais",
        height=800
    )
    
    fig_dashboard.show()
    
    # Tabela de métricas
    print("\n📊 Resumo Executivo:")
    
    metrics_df = pd.DataFrame(metrics).T
    metrics_df = metrics_df.round(2)
    
    # Renomear colunas
    metrics_df.columns = [
        'Preço Atual ($)', 'Retorno Total (%)', 'Volatilidade (%)',
        'Preço Máximo ($)', 'Preço Mínimo ($)', 'Volume Médio'
    ]
    
    display(metrics_df)
    
    # Insights automáticos
    print("\n💡 Insights Automáticos:")
    
    best_performer = metrics_df['Retorno Total (%)'].idxmax()
    worst_performer = metrics_df['Retorno Total (%)'].idxmin()
    most_volatile = metrics_df['Volatilidade (%)'].idxmax()
    least_volatile = metrics_df['Volatilidade (%)'].idxmin()
    
    print(f"  🏆 Melhor performance: {best_performer} ({metrics_df.loc[best_performer, 'Retorno Total (%)']:.2f}%)")
    print(f"  📉 Pior performance: {worst_performer} ({metrics_df.loc[worst_performer, 'Retorno Total (%)']:.2f}%)")
    print(f"  📈 Mais volátil: {most_volatile} ({metrics_df.loc[most_volatile, 'Volatilidade (%)']:.2f}%)")
    print(f"  📊 Menos volátil: {least_volatile} ({metrics_df.loc[least_volatile, 'Volatilidade (%)']:.2f}%)")

else:
    print("❌ Não há dados disponíveis para o dashboard executivo")

## 8. 🌐 Código do Dashboard Streamlit

Vamos gerar o código para um dashboard web interativo.

In [ ]:
# Gerar código do dashboard Streamlit
print("🌐 Gerando código do Dashboard Streamlit...")

streamlit_code = '''
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import sys
import os

# Configuração da página
st.set_page_config(
    page_title="📊 Big Data Finance Dashboard",
    page_icon="📈",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Título principal
st.title("📊 Big Data Finance - Dashboard Interativo")
st.markdown("---")

# Sidebar
st.sidebar.header("🔧 Configurações")

# Carregar dados (simulado)
@st.cache_data
def load_data():
    # Aqui você carregaria os dados reais
    dates = pd.date_range(start='2023-01-01', end='2024-01-01', freq='D')
    symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN']
    
    data = []
    for symbol in symbols:
        base_price = np.random.uniform(100, 300)
        for date in dates:
            price_change = np.random.normal(0, 0.02)
            base_price *= (1 + price_change)
            
            data.append({
                'date': date,
                'symbol': symbol,
                'close': base_price,
                'volume': np.random.randint(1000000, 10000000)
            })
    
    return pd.DataFrame(data)

# Carregar dados
df = load_data()

# Seleção de símbolos
selected_symbols = st.sidebar.multiselect(
    "📈 Selecione os ativos:",
    options=df['symbol'].unique(),
    default=df['symbol'].unique()[:3]
)

# Filtrar dados
df_filtered = df[df['symbol'].isin(selected_symbols)]

# Layout em colunas
col1, col2, col3 = st.columns(3)

with col1:
    st.metric(
        label="📊 Total de Registros",
        value=f"{len(df_filtered):,}"
    )

with col2:
    st.metric(
        label="📈 Ativos Selecionados",
        value=len(selected_symbols)
    )

with col3:
    st.metric(
        label="📅 Período",
        value=f"{(df['date'].max() - df['date'].min()).days} dias"
    )

st.markdown("---")

# Gráfico de preços
st.subheader("📈 Evolução dos Preços")

fig_prices = px.line(
    df_filtered,
    x='date',
    y='close',
    color='symbol',
    title='Evolução dos Preços'
)

st.plotly_chart(fig_prices, use_container_width=True)

# Layout em duas colunas
col1, col2 = st.columns(2)

with col1:
    st.subheader("📊 Volume de Negociação")
    
    fig_volume = px.bar(
        df_filtered.groupby('symbol')['volume'].mean().reset_index(),
        x='symbol',
        y='volume',
        title='Volume Médio por Ativo'
    )
    
    st.plotly_chart(fig_volume, use_container_width=True)

with col2:
    st.subheader("🔗 Matriz de Correlação")
    
    # Pivot para correlação
    pivot_data = df_filtered.pivot(index='date', columns='symbol', values='close')
    corr_matrix = pivot_data.corr()
    
    fig_corr = px.imshow(
        corr_matrix,
        title='Correlação entre Ativos',
        color_continuous_scale='RdBu_r'
    )
    
    st.plotly_chart(fig_corr, use_container_width=True)

# Tabela de dados
st.subheader("📋 Dados Detalhados")
st.dataframe(df_filtered.tail(20), use_container_width=True)

# Rodapé
st.markdown("---")
st.markdown(
    "**Desenvolvido por:** Ricardo Areas & Equipe Big Data Finance | "
    "**Projeto:** Sistema de Análise Financeira com Big Data"
)
'''

# Salvar código do Streamlit
streamlit_file = '../src/streamlit_dashboard.py'
with open(streamlit_file, 'w', encoding='utf-8') as f:
    f.write(streamlit_code)

print(f"✅ Código do Streamlit salvo em: {streamlit_file}")
print("\n🚀 Para executar o dashboard:")
print(f"   cd {os.path.dirname(streamlit_file)}")
print(f"   streamlit run {os.path.basename(streamlit_file)}")

# Mostrar preview do código
print("\n📝 Preview do código gerado:")
print(streamlit_code[:500] + "...")

## 9. 📊 Relatório de Visualizações

Vamos gerar um relatório final com todas as visualizações criadas.

In [ ]:
# Gerar relatório final
print("📊 RELATÓRIO FINAL - VISUALIZAÇÕES INTERATIVAS")
print("=" * 55)

report = {
    'data_loaded': not df_stocks.empty,
    'total_records': len(df_stocks) if not df_stocks.empty else 0,
    'symbols_analyzed': list(df_stocks['symbol'].unique()) if not df_stocks.empty else [],
    'date_range': {
        'start': df_stocks['date'].min().strftime('%Y-%m-%d') if not df_stocks.empty else 'N/A',
        'end': df_stocks['date'].max().strftime('%Y-%m-%d') if not df_stocks.empty else 'N/A'
    },
    'ml_results_available': bool(ml_results),
    'visualizations_created': []
}

# Lista de visualizações criadas
if not df_stocks.empty:
    report['visualizations_created'].extend([
        '📈 Gráfico de linha de preços',
        '🕯️ Gráfico candlestick',
        '📊 Comparação multi-ativos',
        '🔗 Matriz de correlação',
        '📈 Análise de volatilidade',
        '📊 Box plot de volatilidade',
        '📊 Dashboard executivo'
    ])

if ml_results:
    if 'regression' in ml_results:
        report['visualizations_created'].append('🤖 Performance modelos de regressão')
    if 'classification' in ml_results:
        report['visualizations_created'].append('🎯 Performance modelos de classificação')
    if 'feature_importance' in ml_results:
        report['visualizations_created'].append('🔍 Importância das features')
    if 'sentiment' in ml_results:
        report['visualizations_created'].extend([
            '💭 Distribuição de sentimentos',
            '📰 Sentimentos por texto'
        ])

# Adicionar dashboard Streamlit
report['visualizations_created'].append('🌐 Dashboard Streamlit (código gerado)')

print(f"\n📊 RESUMO DO RELATÓRIO:")
print(f"  • Dados carregados: {'✅ Sim' if report['data_loaded'] else '❌ Não'}")
print(f"  • Total de registros: {report['total_records']:,}")
print(f"  • Símbolos analisados: {len(report['symbols_analyzed'])}")
print(f"  • Período: {report['date_range']['start']} até {report['date_range']['end']}")
print(f"  • Resultados ML disponíveis: {'✅ Sim' if report['ml_results_available'] else '❌ Não'}")
print(f"  • Visualizações criadas: {len(report['visualizations_created'])}")

print(f"\n📊 VISUALIZAÇÕES CRIADAS:")
for i, viz in enumerate(report['visualizations_created'], 1):
    print(f"  {i:2d}. {viz}")

print(f"\n🎯 FUNCIONALIDADES DEMONSTRADAS:")
features = [
    '📈 Gráficos interativos com Plotly',
    '🔧 Dashboards personalizáveis',
    '📊 Visualizações financeiras especializadas',
    '🤖 Integração com resultados de ML',
    '💭 Análise visual de sentimentos',
    '📊 Métricas executivas automatizadas',
    '🌐 Dashboard web com Streamlit',
    '📋 Relatórios interativos',
    '🎨 Visualizações responsivas',
    '📱 Interface amigável ao usuário'
]

for i, feature in enumerate(features, 1):
    print(f"  {i:2d}. {feature}")

print(f"\n🚀 PRÓXIMOS PASSOS:")
next_steps = [
    '🌐 Executar dashboard Streamlit: streamlit run ../src/streamlit_dashboard.py',
    '📊 Implementar atualizações em tempo real',
    '🔧 Adicionar mais tipos de gráficos',
    '📱 Otimizar para dispositivos móveis',
    '🎨 Personalizar temas e estilos',
    '📈 Integrar com APIs de dados em tempo real',
    '🔐 Implementar autenticação e autorização',
    '📊 Adicionar exportação de relatórios',
    '🎯 Criar alertas e notificações',
    '📋 Implementar filtros avançados'
]

for i, step in enumerate(next_steps, 1):
    print(f"  {i:2d}. {step}")

# Salvar relatório
import json
report_file = '../data/results/visualization_report.json'
os.makedirs(os.path.dirname(report_file), exist_ok=True)

with open(report_file, 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print(f"\n💾 Relatório salvo em: {report_file}")
print(f"\n🎉 Notebook de Visualizações concluído com sucesso!")
print(f"📊 Todas as funcionalidades de dashboard foram demonstradas")

---

## 🎯 Resumo do Notebook

Este notebook demonstrou:

### ✅ **Funcionalidades Implementadas:**
- 📈 **Gráficos Interativos**: Plotly para visualizações dinâmicas
- 🕯️ **Gráficos Financeiros**: Candlestick, OHLC, volume
- 📊 **Análise Comparativa**: Multi-ativos, correlações
- 📈 **Análise de Volatilidade**: Temporal e comparativa
- 🤖 **Visualização de ML**: Performance de modelos, importância
- 💭 **Análise de Sentimentos**: Distribuição, categorização
- 📊 **Dashboard Executivo**: Métricas principais, insights
- 🌐 **Dashboard Web**: Código Streamlit gerado

### 🛠️ **Tecnologias Utilizadas:**
- **Plotly**: Gráficos interativos e dashboards
- **Pandas**: Manipulação e análise de dados
- **NumPy**: Computação numérica
- **Streamlit**: Dashboard web interativo
- **Matplotlib/Seaborn**: Visualizações complementares

### 📊 **Tipos de Gráficos Criados:**
- 📈 Gráficos de linha para evolução temporal
- 🕯️ Gráficos candlestick para análise técnica
- 📊 Gráficos de barras para comparações
- 🔗 Heatmaps para correlações
- 📊 Box plots para distribuições
- 🥧 Gráficos de pizza para proporções
- 📈 Subplots para análises combinadas

### 🎯 **Próximos Passos:**
1. **Executar Dashboard**: `streamlit run ../src/streamlit_dashboard.py`
2. **Integração Real-time**: Conectar com APIs de dados ao vivo
3. **Personalização**: Temas, cores e layouts customizados
4. **Mobile**: Otimização para dispositivos móveis
5. **Alertas**: Sistema de notificações automáticas

---

**👥 Desenvolvido por:** Ricardo Areas & Equipe Big Data Finance  
**📅 Data:** 2024  
**🎯 Objetivo:** Demonstrar capacidades de visualização interativa do sistema